In [16]:
from collections import defaultdict
import pandas as pd

from surprise.model_selection import KFold, cross_validate

from surprise import KNNBasic

from surprise import SVD
from surprise import SVDpp
from surprise import  NMF
from surprise import NormalPredictor, BaselineOnly
from surprise import SlopeOne
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


from model.data_interface.data import *

In [17]:
data = load_data(data_path="../data/dicoding_user_item_rating.gzip")
print(data.head())

data_input = data.loc[:, ["user_id", "course_id", "rating"]]
surprise_input = reader_data(data=data_input,
                             cols=["user_id", "course_id", "rating"],
                             scale=True,
                             model="surprise")

Index(['user_id', 'course_id', 'graduated_at', 'rating'], dtype='object')
   user_id  course_id        graduated_at  rating
0   623699         14 2020-03-10 11:45:50       5
1   406371         14 2020-03-10 08:44:09       4
2     1946         14 2020-03-11 13:24:10       5
3   186713         14 2020-03-11 17:36:04       4
4   462580         14 2020-03-11 08:43:36       4


In [18]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [19]:
def precision_recall_at_k(predictions, k=10, threshold=4.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [20]:
trainset = surprise_input.build_full_trainset()

testset  = trainset.build_anti_testset()

In [21]:
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.

predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

In [22]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    if uid == int("2447675"):
        print(uid, [iid for (iid, _) in user_ratings])

2447675 [276, 418, 510, 615, 332, 153, 133, 605, 413, 271]


In [23]:
kf = KFold(n_splits=5)
algo = SVD()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8751018989162754
0.8925061832501652
0.8734352850724397
0.8906646013034617
0.8767182857004815
0.893536768543412
0.8756636103289778
0.8935293980146338
0.8724251880300857
0.8902369998247339


In [24]:
kf = KFold(n_splits=5)
algo = SVDpp()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8721246224535706
0.889829895633467
0.8714957899950483
0.8881449620934595
0.8720755154042158
0.889325961235333
0.8727464242598686
0.88930001866526
0.8740997229916903
0.890687334799102


In [25]:
 
kf = KFold(n_splits=5)
algo = NormalPredictor()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.7614404401032892
0.7358152578013764
0.7631288550501157
0.7382281378884116
0.7604661368568869
0.7363160348630485
0.7649081799002688
0.7403979297696934
0.7636483318883861
0.7387230352918868


In [26]:
 
kf = KFold(n_splits=5)
algo = BaselineOnly()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

Estimating biases using als...
0.8771127716306627
0.8958195570136941
Estimating biases using als...
0.8758352311318875
0.8943662433827841
Estimating biases using als...
0.8759509683824545
0.894954980767317
Estimating biases using als...
0.8763195012101107
0.8953530897278791
Estimating biases using als...
0.8760996456118411
0.8947649329684976


In [27]:

kf = KFold(n_splits=5)
algo = NMF()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.566140953570685
0.5253649977962903
0.561277382782475
0.5192664192712233
0.569766603369949
0.5301650445690536
0.5790559867585849
0.5387818620499524
0.5860986187601709
0.5476407250883569


In [28]:

kf = KFold(n_splits=5)
algo = NMF()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.5651623451904085
0.5230425072783135
0.6067869065059198
0.5726682231287284
0.5991862088891025
0.5604202829427943
0.5882707737193513
0.547676380616361
0.5726671476671478
0.5308154771448425


In [29]:

kf = KFold(n_splits=5)
algo = CoClustering()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8331387511434215
0.8407322944014726
0.8410244051348799
0.849055768958044
0.8421368917501432
0.8516160175898577
0.8403589188322106
0.8488355493426476
0.8383118751348634
0.8460052871996337


In [30]:

kf = KFold(n_splits=5)
algo = CoClustering()

data = surprise_input

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4.5)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8417096447679188
0.8500022400520064
0.8377023273632794
0.8455000284171343
0.839436044135677
0.8478397393557616
0.8418786598828839
0.8519022101249469
0.8439096140689779
0.8530477195145798
